# Spotify Data Preparing
using my own spoify data
_______
## Import Library

In [1]:
import pandas as pd
import numpy as np

_____
## Import Data

In [2]:
# import streaming history data from spotify
df_stream0 = pd.read_json('StreamingHistory0.json')
df_stream1 = pd.read_json('StreamingHistory1.json')
df_stream2 = pd.read_json('StreamingHistory2.json')

# merge all stream dataframe
df_stream = pd.concat([df_stream0, df_stream1, df_stream2])

df_stream.head()

,endTime,artistName,trackName,msPlayed
0,2020-09-20 19:26,BOL4,To My Youth,220260
1,2020-09-21 10:29,BTS,Euphoria,228615
2,2020-09-21 10:32,V,Sweet Night,214258
3,2020-09-21 10:36,BTS,Love Maze,221529
4,2020-09-21 10:39,Lauv,Who (feat. BTS),180413


In [3]:
# import library data from spotify
df_library = pd.read_json('YourLibrary.json')

df_library.head()

,artist,album,track,uri
0,Maurice Ravel,Lalo: Symphonie Espagnole/Saint-Saens: Vln Cto...,"Ravel: Tzigane, M. 76",spotify:track:5UGtJ6cy1pk0myJiZ3S8Md
1,César Franck,Virtuoso,Sonata for Violin and Piano in A Major: Allegr...,spotify:track:2yuxVUVg9E7r0V4wv2STyb
2,Arctic Monkeys,AM,Arabella,spotify:track:2LdG9xtEzCho3jCr58M6cq
3,Clara Schumann,Romance – The Piano Music of Clara Schumann,"3 Romances, Op. 11: 1. Andante",spotify:track:6VLzm03YUeXVl3GdGyib6J
4,Ludwig van Beethoven,"Beethoven: Piano Sonatas, Vol.3","Sonata No. 14 ""Moonlight"" in C-Sharp Minor"", O...",spotify:track:3DNRdudZ2SstnDCVKFdXxG


______
## Data Preparation

In [4]:
# create 'uniqueID' column by combining 'artistName' and 'trackName' column in df_stream
df_stream['uniqueID'] = df_stream['artistName'] + ':' + df_stream['trackName']

df_stream.head()

,endTime,artistName,trackName,msPlayed,uniqueID
0,2020-09-20 19:26,BOL4,To My Youth,220260,BOL4:To My Youth
1,2020-09-21 10:29,BTS,Euphoria,228615,BTS:Euphoria
2,2020-09-21 10:32,V,Sweet Night,214258,V:Sweet Night
3,2020-09-21 10:36,BTS,Love Maze,221529,BTS:Love Maze
4,2020-09-21 10:39,Lauv,Who (feat. BTS),180413,Lauv:Who (feat. BTS)


In [5]:
# create 'uniqueID' column by combining 'artistName' and 'trackName' column in df_library
df_library['uniqueID'] = df_library['artist'] + ':' + df_library['track']

# add 'track_url' column by split 'url' column
df_library['track_uri'] = df_library['uri'].str.split(':', expand = True)[2]

df_library.head()

,artist,album,track,uri,uniqueID,track_uri
0,Maurice Ravel,Lalo: Symphonie Espagnole/Saint-Saens: Vln Cto...,"Ravel: Tzigane, M. 76",spotify:track:5UGtJ6cy1pk0myJiZ3S8Md,"Maurice Ravel:Ravel: Tzigane, M. 76",5UGtJ6cy1pk0myJiZ3S8Md
1,César Franck,Virtuoso,Sonata for Violin and Piano in A Major: Allegr...,spotify:track:2yuxVUVg9E7r0V4wv2STyb,César Franck:Sonata for Violin and Piano in A ...,2yuxVUVg9E7r0V4wv2STyb
2,Arctic Monkeys,AM,Arabella,spotify:track:2LdG9xtEzCho3jCr58M6cq,Arctic Monkeys:Arabella,2LdG9xtEzCho3jCr58M6cq
3,Clara Schumann,Romance – The Piano Music of Clara Schumann,"3 Romances, Op. 11: 1. Andante",spotify:track:6VLzm03YUeXVl3GdGyib6J,"Clara Schumann:3 Romances, Op. 11: 1. Andante",6VLzm03YUeXVl3GdGyib6J
4,Ludwig van Beethoven,"Beethoven: Piano Sonatas, Vol.3","Sonata No. 14 ""Moonlight"" in C-Sharp Minor"", O...",spotify:track:3DNRdudZ2SstnDCVKFdXxG,"Ludwig van Beethoven:Sonata No. 14 ""Moonlight""...",3DNRdudZ2SstnDCVKFdXxG


In [12]:
# create dataframe for tableau visualization
df_tableau = df_stream.copy()

# create 'inLibrary' columm to check if streamed song is in library 
df_tableau['inLibrary'] = np.where(df_tableau['uniqueID'].isin(df_library['uniqueID'].tolist()), 1, 0) # 1 if included and 0 if not included

# left join df_tableau and df_library 
df_tableau = pd.merge(df_tableau, df_library[['album', 'uniqueID', 'track_uri']], how='left', on=['uniqueID'])

df_tableau.head()

,endTime,artistName,trackName,msPlayed,uniqueID,inLibrary,album,track_uri
0,2020-09-20 19:26,BOL4,To My Youth,220260,BOL4:To My Youth,0,NaN,NaN
1,2020-09-21 10:29,BTS,Euphoria,228615,BTS:Euphoria,0,NaN,NaN
2,2020-09-21 10:32,V,Sweet Night,214258,V:Sweet Night,0,NaN,NaN
3,2020-09-21 10:36,BTS,Love Maze,221529,BTS:Love Maze,0,NaN,NaN
4,2020-09-21 10:39,Lauv,Who (feat. BTS),180413,Lauv:Who (feat. BTS),0,NaN,NaN


_____
## Export df_tableau to csv


In [14]:
# save df_tableau as csv files to load into Tableau
df_tableau.to_csv('MySpotifyDataTable.csv')

_____
## Create genre dataframe using Spotify’s 

source: https://stmorse.github.io/journal/spotify-api.html

In [15]:
import requests

CLIENT_ID = 'YOUR CLIENT ID'
CLIENT_SECRET = 'YOUR CLIENT SECRET'

In [16]:
# authentication URL
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [17]:
# used for authenticating all API calls
headers = {'Authorization': 'Bearer {token}'.format(token=access_token)}

In [18]:
# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

In [20]:
# create blank dictionary to store track URI, artist URI, and genres
dict_genre = {}

# convert track_uri column to an iterable list
track_uris = df_library['track_uri'].to_list()

# loop through track URIs and pull artist URI using the API, then use artist URI to pull genres associated with that artist store all these in a dictionary
for t_uri in track_uris:
    
    dict_genre[t_uri] = {'artist_uri': "", "genres":[]}
    
    r = requests.get(BASE_URL + 'tracks/' + t_uri, headers=headers)
    r = r.json()
    a_uri = r['artists'][0]['uri'].split(':')[2]
    dict_genre[t_uri]['artist_uri'] = a_uri
    
    s = requests.get(BASE_URL + 'artists/' + a_uri, headers=headers)
    s = s.json()
    dict_genre[t_uri]['genres'] = s['genres']

In [21]:
# convert dictionary into dataframe with track_uri as the first column
df_genre = pd.DataFrame.from_dict(dict_genre, orient='index')
df_genre.insert(0, 'track_uri', df_genre.index)
df_genre.reset_index(inplace=True, drop=True)

df_genre.head()

,track_uri,artist_uri,genres
0,5UGtJ6cy1pk0myJiZ3S8Md,17hR0sYHpx7VYTMRfFUOmY,"[classical, impressionism, neoclassicism, post..."
1,2yuxVUVg9E7r0V4wv2STyb,1C3sffOOvQNUwg4YIsvKqy,"[classical, french romanticism, late romantic ..."
2,2LdG9xtEzCho3jCr58M6cq,7Ln80lUS6He07XvHI8qqHH,"[garage rock, permanent wave, rock, sheffield ..."
3,6VLzm03YUeXVl3GdGyib6J,2yzaWNFV3cxmcRZtwtr5WC,"[early romantic era, german romanticism]"
4,3DNRdudZ2SstnDCVKFdXxG,2wOqMjp9TyABvtHdOSOTUS,"[classical, classical era, early romantic era,..."


In [22]:
df_genre_expanded = df_genre.explode('genres')
df_genre_expanded.head()

,track_uri,artist_uri,genres
0,5UGtJ6cy1pk0myJiZ3S8Md,17hR0sYHpx7VYTMRfFUOmY,classical
0,5UGtJ6cy1pk0myJiZ3S8Md,17hR0sYHpx7VYTMRfFUOmY,impressionism
0,5UGtJ6cy1pk0myJiZ3S8Md,17hR0sYHpx7VYTMRfFUOmY,neoclassicism
0,5UGtJ6cy1pk0myJiZ3S8Md,17hR0sYHpx7VYTMRfFUOmY,post-romantic era
1,2yuxVUVg9E7r0V4wv2STyb,1C3sffOOvQNUwg4YIsvKqy,classical


In [23]:
# save df_genre_expanded as csv files 
df_genre_expanded.to_csv('GenresExpandedTable.csv')